In [0]:
import dlt
from pyspark.sql.functions import *

The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-6823359354773124>, line 1
----> 1 import dlt
      2 from pyspark.sql import functions as F

ModuleNotFoundError: No module named 'dlt'

In [0]:
@dlt.table(
    name="BRONZE",
    comment="Raw pipe-delimited data loaded into the Bronze layer."
)
def bronze_load():
    df = (
        spark.read.format("csv")
        .option("delimiter", "|")        # Pipe-delimited CSV
        .option("header", True)
        .option("inferSchema", True)
        .load("/Volumes/workspace/damg7370/datastore/material_master_1k.csv")
    )
    return df

In [0]:
import dlt
from pyspark.sql.functions import col, trim, current_timestamp

@dlt.expect_or_drop("valid_cost", "unit_cost > 0")  # Drop invalid cost rows
@dlt.expect("valid_lead_time", "lead_time_days >= 0")
@dlt.expect("valid_safety_stock", "safety_stock >= 0")
@dlt.table(
    name="SILVER",
    comment="Single Silver layer — cleaned, standardized, and validated material master data."
)
def silver_material_master():
    # Read data from Bronze layer
    df = dlt.read("BRONZE")

    # Trim whitespace from all string columns
    df = df.select([trim(col(c)).alias(c) for c in df.columns])

    # Drop records with null critical fields
    df = df.dropna(subset=["material_id", "material_name"])

    # Convert datatypes for numeric columns
    df = (
        df.withColumn("unit_cost", col("unit_cost").cast("double"))
          .withColumn("lead_time_days", col("lead_time_days").cast("int"))
          .withColumn("safety_stock", col("safety_stock").cast("int"))
          .withColumn("reorder_level", col("reorder_level").cast("int"))
          .withColumn("load_timestamp", current_timestamp())
    )

    # Return cleaned and validated dataframe
    return df
